In [61]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import mysql.connector
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


# Connect to the MySQL database
db = mysql.connector.connect(
  host='localhost',
  user='root',
  password='',
  database='college_placement'
)

# Create a cursor object
cursor = db.cursor()

# Retrieve the data from the database
cursor.execute("SELECT * FROM college_place")
data = cursor.fetchall()

# Create a pandas dataframe from the data
df = pd.DataFrame(data, columns=["Age", "Gender", "Stream", "Internships", "CGPA", "Hostel", "HistoryOfBacklogs", "PlacedOrNot"])

# Preprocess the data
X = df.drop(["PlacedOrNot"], axis=1)
y = df["PlacedOrNot"]

X = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

# Train the model
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

# Create the GUI
root = tk.Tk()
root.title("College Placement Prediction")
root.geometry("800x600")

# Create a title label
title_label = tk.Label(root, text="College Placement Prediction Tool", font=("Arial", 30, "bold", "italic"), fg="#00698f", bg="darkseagreen")
title_label.pack(pady=20, fill="x")

# Create a frame for the input fields
input_frame = tk.Frame(root, bg="darkseagreen")
input_frame.pack(fill="both", expand=True, padx=20, pady=20)

labels = ["Age", "Gender", "Stream", "Internships (0:No, 1:Yes)", "CGPA", "Hostel (0:No, 1:Yes)", "HistoryOfBacklogs"]
entries = []

for i, label in enumerate(labels):
    tk.Label(input_frame, text=label, bg="darkseagreen", font=("Arial", 14)).grid(row=i, column=0, padx=10, pady=10)
    
    if label == "Stream":
        stream_var = tk.StringVar()
        stream_var.set("Select Stream")  # default value
        stream_menu = tk.OptionMenu(input_frame, stream_var, "Civil", "Computer Science", "Electrical", "Electronics and Communication", "Information Technology", "Mechanical")
        stream_menu.config(width=20)
        stream_menu.grid(row=i, column=1, padx=10, pady=10)
        entries.append(stream_var)
        
    elif label == "Gender":
        gender_var = tk.StringVar()
        gender_var.set("Male")  # default value
        tk.Radiobutton(input_frame, text="Male", variable=gender_var, value="Male", bg="lightblue", fg="black", selectcolor="blue", font=("Arial", 14), indicatoron=0, width=10, height=2).grid(row=i, column=1, padx=10, pady=10)
        tk.Radiobutton(input_frame, text="Female", variable=gender_var, value="Female", bg="pink", fg="black", selectcolor="red", font=("Arial", 14), indicatoron=0, width=10, height=2).grid(row=i, column=2, padx=10, pady=10)
        entries.append(gender_var)
    else:
        entry = tk.Entry(input_frame, width=20, font=("Arial", 14))
        entry.grid(row=i, column=1, padx=10, pady=10)
        entries.append(entry)

# Create the predict button
predict_button = tk.Button(input_frame, text="Predict", command=lambda: predict_collegePlacement(predict_button), bg="green", fg="white", font=("Arial", 14, "bold"))
predict_button.grid(row=len(labels), column=0, columnspan=3, padx=10, pady=10)

def predict_collegePlacement(predict_button):
    data = []
    for entry in entries:
        if isinstance(entry, tk.StringVar):
            data.append(entry.get())
        else:
            data.append(entry.get())
    
    if any(not x for x in data):
        messagebox.showerror("Error", "Please fill all the fields")
    else:
        cgpa = float(data[4])
        internship = int(data[3])
        
        if cgpa >= 8 and internship == 1 and int(data[6]) == 0:
            messagebox.showinfo("Placement Status", "You are likely to be placed.")
        else:
            newdf = pd.DataFrame([data], columns=labels)
            newdf = pd.get_dummies(newdf)
            
            missing_cols = set(X_train.columns) - set(newdf.columns)
                       
            for c in missing_cols:
                newdf[c] = 0
            newdf = newdf[X_train.columns]
            yp = model.predict(newdf)
                                
            if yp[0] == 1:
                messagebox.showinfo("Placement Status", "You are likely to be placed.")
            else:
                messagebox.showinfo('Placement Status','You are not likely to be placed.')
            
            # Store the input data in the database if it's not present in the CSV file
            cursor.execute("SELECT * FROM college_place WHERE Age = %s AND Gender = %s AND Stream = %s AND Internships = %s AND CGPA = %s AND Hostel = %s AND HistoryOfBacklogs = %s", 
                           (data[0], data[1], data[2], data[3], data[4], data[5], data[6]))
            if cursor.fetchone() is None:
                cursor.execute("INSERT INTO college_place (Age, Gender, Stream, Internships, CGPA, Hostel, HistoryOfBacklogs, PlacedOrNot) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", 
                               (data[0], data[1], data[2], data[3], data[4], data[5], data[6], yp[0]))
                db.commit()
            
        predict_button.config(state='disabled')

root.mainloop()

In [60]:
print(model.score(X_test, y_test))

1.0
